<a href="https://colab.research.google.com/github/themasterprogram/B-FacialEmotionRecognition-Talha/blob/main/FERCOMMITT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils import plot_model
from keras.models import Model

from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/Drive')

Mounted at /content/Drive


In [3]:
%cd /content/Drive/MyDrive

/content/Drive/MyDrive


In [4]:
%cd /content/Drive/MyDrive/fer/fer2013/fer2013

/content/Drive/MyDrive/fer/fer2013/fer2013


In [5]:
X_train = []
y_train = []
X_test = []
y_test = []
df = pd.read_csv("fer2013.csv")
for index, row in df.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k))
        y_test.append(row['emotion'])

X_train = np.array(X_train, dtype = 'uint8')
y_train = np.array(y_train, dtype = 'uint8')
X_test = np.array(X_test, dtype = 'uint8')
y_test = np.array(y_test, dtype = 'uint8')

In [7]:
import keras
from keras.utils import to_categorical
y_train= to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [8]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [9]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 10,
    horizontal_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode = 'nearest')
testgen = ImageDataGenerator(rescale=1./255)
datagen.fit(X_train)
batch_size = 64

In [10]:
train_flow = datagen.flow(X_train, y_train, batch_size=batch_size)
test_flow = testgen.flow(X_test, y_test, batch_size=batch_size)

In [18]:
def FER_Model(input_shape=(48,48,1)):
    # first input model
    visible = Input(shape=input_shape, name='input')
    num_classes = 7
    #the 1-st block
    conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='valid', name = 'conv1_1')(visible)
    #After 1st convolution the feature map is 46x46
    conv1_1 = BatchNormalization()(conv1_1)
    pool1_1 = MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid')(conv1_1)
    #After Max Pooling Layer the feature map becomes 23x23

    #2nd Block:
    conv2 = Conv2D(128, kernel_size = 3, activation = 'relu', padding = 'valid', name = 'conv2')(pool1_1)
    #After this convolution layer feature map becomes 21x21
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size = (2,2),strides = 1,padding = 'valid')(conv2)
    #After this maxpooling layer feature map becomes 20x20

    #3rd Block:
    conv3 = Conv2D(256, kernel_size=3, activation='relu', padding='valid', name = 'conv3')(pool2)
    #After this conv layer feature map becomes 18x18
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2),strides = 2, padding = 'valid')(conv3)
    #After this pooling layer feature map becomes 9x9
    drop1 = Dropout(0.3, name = 'drop1')(pool3)
    #the 4th block
    conv4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv4')(drop1)
    #After this conv layer feature map becomes 7x7
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size = (2,2), strides = 1, padding = 'valid')(conv4)
    #After this pooling layer feature map becomes 6x6
    conv5 = Conv2D(1024, kernel_size=3, activation='relu', padding='same', name = 'conv5')(pool4)
    #After this conv layer feature map becomes 4x4
    conv5 = BatchNormalization()(conv5)
    pool5 = MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'valid')(conv5)
    #After this pooling layer feature map becomes 2x2
    drop2 = Dropout(0.3, name = 'drop2')(pool5)


    flatten = Flatten(name = 'flatten')(drop2)
    #I chose 2048 as number of neurons in this layer because of 2*1024, 2x2 was the size of the latest feature map, 1024 was the number of feature maps/filters
    Dense1 = Dense(2048,activation = 'relu',name = 'Dense1' )(flatten)
    Dense2 = Dense(1024, activation = 'relu', name = 'Dense2')(Dense1)
    Dense3 = Dense(512, activation = 'relu', name = 'Dense3')(Dense2)
    Dense4 = Dense(256, activation = 'relu', name = 'Dense4')(Dense3)
    ouput = Dense(num_classes, activation='softmax', name = 'output')(Dense4)# create model
    model = Model(inputs =visible, outputs = ouput)
    # summary layers
    print(model.summary())

    return model

In [19]:
model = FER_Model()
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 48, 48, 1)]       0         
                                                                 
 conv1_1 (Conv2D)            (None, 46, 46, 64)        640       
                                                                 
 batch_normalization_14 (Ba  (None, 46, 46, 64)        256       
 tchNormalization)                                               
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 23, 23, 64)        0         
 ng2D)                                                           
                                                                 
 conv2 (Conv2D)              (None, 21, 21, 128)       73856     
                                                                 
 batch_normalization_15 (Ba  (None, 21, 21, 128)       512 

In [ ]:
num_epochs = 100
history = model.fit_generator(train_flow,
                    steps_per_epoch=len(X_train) / batch_size,
                    epochs=num_epochs,
                    verbose=1,
                    validation_data=test_flow,
                    validation_steps=len(X_test) / batch_size)

<ipython-input-20-37e6bcd0ecfb>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_flow,


Epoch 1/100
 35/448 [=>............................] - ETA: 43:48 - loss: 1.9934 - accuracy: 0.2375

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")